In [12]:
import sqlite3
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [9]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
#url of the page we want to scrape
symbol = "AIMCG"
url = 'https://www.set.or.th/th/market/product/stock/quote/'+symbol+'/financial-statement/company-highlights'

# initiating the webdriver. Parameter includes the path of the webdriver.
driver = webdriver.Chrome('./chromedriver') 
driver.get(url) 

# this is just to ensure that the page is loaded
time.sleep(3) 

html = driver.page_source

data = pd.read_html(html)
print()
AOT_financial_statement=[]
table = data[0]
table=table.dropna()
column_name = table.columns.tolist()
for cols in column_name[1:]:
    year = [cols[-4:]]
    statement = table[cols].tolist()
    if '-' in statement:
        for i in range(len(statement)):
            if statement[i] == '-':
                statement[i] = "null"
    AOT_financial_statement.append(year+statement)
print(AOT_financial_statement)


driver.close() # closing the webdriver

C:\Users\Krittapon\AppData\Local\Temp\ipykernel_22680\3966277087.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')



[['2562', '3333.08', '336.75', 'null', 'null', '377.67', '119.20', 'null', 'null', 'null', 'null', 'null'], ['2563', '3421.13', '438.91', 'null', 'null', '388.94', '-68.14', 'null', 'null', 'null', 'null', 'null'], ['2564', '3344.14', '481.99', 'null', 'null', '362.19', '-296.42', 'null', 'null', 'null', 'null', 'null'], ['2565', '3301.67', '412.07', 'null', 'null', '247.83', '2.00', 'null', 'null', 'null', 'null', 'null']]
